In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install pymorphy2 stop_words annoy

In [ ]:
from tqdm import tqdm_notebook, tqdm, notebook
import string
import shutil
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import FastText
import pickle
import numpy as np
import re
import pickle

# Модель ответа на общие вопросы

In [ ]:
# Загружаем файл содержащий вопросы и варианты ответа на них.
!wget -O Otvety.txt 'https://drive.google.com/uc?export=download&confirm=yes&id=1DQL9ybca4USImUDaxxHmkIZNmClKBtKG'

--2024-01-10 07:25:18--  https://drive.google.com/uc?export=download&confirm=yes&id=1DQL9ybca4USImUDaxxHmkIZNmClKBtKG
Resolving drive.google.com (drive.google.com)... 74.125.195.101, 74.125.195.113, 74.125.195.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-4o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qd9lp24r0sq6g02r81mjgfc4oroumesb/1704871500000/08017369226157329473/*/1DQL9ybca4USImUDaxxHmkIZNmClKBtKG?e=download&uuid=74d13225-cafd-410e-b57b-dc6226352d7c [following]
--2024-01-10 07:25:18--  https://doc-0k-4o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qd9lp24r0sq6g02r81mjgfc4oroumesb/1704871500000/08017369226157329473/*/1DQL9ybca4USImUDaxxHmkIZNmClKBtKG?e=download&uuid=74d13225-cafd-410e-b57b-dc6226352d7c
Resolving doc-0k-4o-docs.googleusercontent.com (doc-0k-4o-docs.googleusercontent.com)... 142.250.9

In [ ]:
end = ['?', '.', '!', ]
def check_sign(obj:str):
    obj = ' '.join(obj.strip().split())
    for n, el in enumerate(obj[::-1]):
        if el in end:
            finish = n
        elif el == ' ':
            continue
        else:
            break
    return obj[:-finish] if finish > 0 else obj[:]

'''
Функция для предобработки файла с вопросами и ответами. На вход подается текстовый файл содержащий строки:
с вопросами которые начинается со строки "---", далее следует срока с вопросом, и строки с ответами.
Ответов может быть несколько.
'''
# Определяем флаг
written = False
# Задаем переменную, где будет храниться текст вопроса.
question = None

# Открываем итоговый файл для записи
with open("prepared_answers.txt", "w") as res:
  # Открываем исходный файл на чтение
  with open('Otvety.txt', 'r')as source:
    for line in notebook.tqdm(source):
      # Если новый или первый вопрос
      if line.startswith("---"):
          # Меняем флаг
          written = True
          # Если это новый вопрос записываем предыдущий вопрос и ответ на него как последний элемент списка ответов.
          if question and answer:
              # Берем последний элемент из списка вопросов
              spam = answer[-1].replace('\t', ' ')
              # Убираем HTML тэги
              spam = re.sub(r'(<.{,4}>)', '', spam)
              # Проверяем что ответ заканчивается только одним знаком препинания ['?', '.', '!', ]
              spam = check_sign(spam)
              # Записываем строку
              res.write(question + '\t' + spam + '\n')
              # И задаем что переменная вопрос пустая
              question = None
          # Переходим к следующей строке
          continue
      # Если флаг True (была срока задающая начало вопроса)
      if written:
          # Вопрос равен этой строке
          question = line.strip().replace('\t', ' ')
          # Создаём список для ответов
          answer = []
          # Меняем флаг, текст вопроса уже есть.
          written = False
          continue
      # Если нет флага и есть вопрос, заполняем список ответов
      if not written and question:
          answer.append(line)
          continue

0it [00:00, ?it/s]

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru")).difference(['не', 'нет'])
exclude = set(string.punctuation).union(['—'])

In [ ]:
def preprocess_txt(line):
    # Убираем HTML тэги
    spls = re.sub(r'(<.{,4}>)', '', line)
    # Ищем цифры, разделяем их пробелами
    spls = [i + ' ' if i.isdigit() else i for i in spls.strip()]
    # Убираем знаки препинания
    spls = ''.join(i for i in spls if i not in exclude).split()
    # spls = [i.lower() for i in spls]
    # Приводим к нормальной форме
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    # Убираем стоп-слова
    spls = [i for i in spls if i not in sw]
    return spls

In [ ]:
sentences, c = [], 0
with open("Otvety.txt", "r") as fin:
    for line in notebook.tqdm(fin):
        if len(line) > 4:
            spam = preprocess_txt(line)
            sentences.append(spam)
            c += 1
            if c > 300000:
                break

0it [00:00, ?it/s]

In [ ]:
modelFT = FastText(sentences=sentences, vector_size=450, min_count=1, window=5, workers=2)

In [ ]:
ft_index = annoy.AnnoyIndex(450 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in notebook.tqdm(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])

        vector_ft = np.zeros(450)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)

        counter += 1

        if counter > 300000:
            break

ft_index.build(15)

0it [00:00, ?it/s]

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
# сохранение модели, дерева индексов, словаря индексов
path = '/content/drive/MyDrive/bot/'
shutil.copy('prepared_answers.txt', path + 'prepared_answers.txt')
modelFT.save(path + 'ft_model')
ft_index.save(path + 'speaker.ann')
with open(path + "index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

'/content/drive/MyDrive/bot/prepared_answers.txt'

# Генерация текста


In [ ]:
!pip install accelerate -U

In [ ]:
# Загружаем текст
from google.colab import files
files.upload()

In [ ]:
# Считываем файл создаем список строк; разделить - пустая строка
def make_corpus(path:str):
    with open(path, 'r', encoding='utf-8')as f:
        data =f.read()
    data = data.replace('\ufeff', '')
    return data.split(sep='\n\n')

In [ ]:
# Создаем корпусы текстов
corpus_1 = make_corpus('wealth_1.txt')
corpus_2 = make_corpus('wealth_2.txt')

In [ ]:
# Запишем измененный текст разделяя части \n
with open('prepare_wealth.txt', 'w', encoding='utf-8')as f:
  for corpus in [corpus_1, corpus_2]:
      for el in corpus:
          spam = el.replace('\n', ' ')
          # f.writelines(spam + '\n')
          f.writelines('<s> ' + spam + ' </s>')
# corpus[:50]
# len(corpus)

In [ ]:
with open('prepare_wealth.txt', 'r')as f:
  for line in f:
    print(line)
    input()

In [ ]:
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
# Создаем датасет
train_dataset = TextDataset(tokenizer=tokenizer, file_path='prepare_wealth.txt', block_size=256)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True,)
# Определяем модель
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [ ]:
# Для тренировки воспользуемся встроенными модулями из библиотеки transformers
training_args = TrainingArguments(
    output_dir="./gpt2-wealth", #Директория для сохранения
    overwrite_output_dir=True, #Перезаписываем данные
    num_train_epochs=25, # Число эпох для тренировки
    per_device_train_batch_size=4, # Размер батча для тренировки
    per_device_eval_batch_size=4,  # Размер батча для теста
    # per_gpu_train_batch_size=4,
    # per_gpu_eval_batch_size=4,
    eval_steps = 400, # Число шагов до оцеки
    save_steps=400, # Число шагов до сохранения модели
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=train_dataset)

In [ ]:
# Запускаем обучение
trainer.train()

In [ ]:
# Сохраняем результаты
trainer.save_model()

In [ ]:
tokenizer.save_pretrained('gpt_wealth')
model.save_pretrained('model_gpt_wealth')

In [ ]:
# Используем обученную модель и токенайзер
tokenizer = AutoTokenizer.from_pretrained("gpt_wealth")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_wealth")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/'

Mounted at /content/drive/


In [ ]:
import shutil

In [ ]:
shutil.copytree('model_gpt_wealth', path + 'model_gpt_wealth')

'/content/drive/MyDrive/model_gpt_wealth'

Тестируем

In [ ]:
prefix = "Что мне мешает стать богатым?"

In [ ]:
tokens = tokenizer(prefix, return_tensors='pt')

In [ ]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens,
    #end_token=end_token_id,
    do_sample=True,
    max_length=size+150,
    repetition_penalty=6.,
    temperature=1.5,
    num_beams=10,)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Что мне мешает стать богатым? - Я боюсь, что это не получится у меня... навсегда.
КАК МЫ ЗАРАБАТЫВАЕМ ДЕНЬГИ?
Деньги - изобретение чисто человеческое. Их можно получить только от других людей. А значит - с помощью них мы добываем деньги. Но для этого нужно убедить их отдать нам свои деньги в обмен на что-то, что мы можем им дать. Согласитесь, кто захочет покупать кота в мешке? Да и зачем ему это надо? Он же не виноват, что у него мало денег. Продавец сам все расстраивает, потому что он не знает, как сделать так, чтобы люди тебе отдали свои деньги в обмен на что-то, что они имеют. Как говорил один миллионер: "


# Разделение вопросов

Идея разделения вопросов по темам следующая:
1. Составляем словарь из слов вопросов на общие темы;
2. Составляем словарь из текста книг (на которых обучалась модель генерации текста)
3. Из множества слов второго словаря убираем слова которые встречаются в первом словаре.
4. Используем получившийся список как набор для определения принадлежности вопроса к модели генерации текста.

In [ ]:
!pip install pymorphy2 stop_words annoy

In [ ]:
from tqdm import tqdm_notebook, tqdm, notebook
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import pickle
from collections import Counter

In [ ]:
# Составляем список текстов общих вопросов
questions, counter = [], 0
with open(path + "prepared_answers.txt", "r") as f:
    for line in notebook.tqdm(f):
        spls = line.split("\t")
        questions.append(preprocess_txt(spls[0]))
        counter += 1
        if counter > 300_000:
          break

0it [00:00, ?it/s]

In [ ]:
# Загружаем тексты книг
from google.colab import files
files.upload()

In [ ]:
# Считываем файл создаем список строк; разделить - пустая строка
def make_corpus(path:str):
    with open(path, 'r', encoding='utf-8')as f:
        data =f.read()
    data = data.replace('\ufeff', '')
    return data.split(sep='\n\n')

In [ ]:
# Создаем корпусы текстов
corpus_1 = make_corpus('wealth_1.txt')
corpus_2 = make_corpus('wealth_2.txt')
corpus_1.extend(corpus_2)

In [ ]:
# Создаем словари получаем список специфических слов
%%time
vocab_general = Counter([i for el in questions for i in el])
vocab_wealth = Counter([i for el in corpus_1 for i in preprocess_txt(el)])
kw1 = [el[0] for el in sorted(vocab_general.items(), key=lambda x: -x[1])[:1000]]
kw2 = set([el[0] for el in sorted(vocab_wealth.items(), key=lambda x: -x[1])[:500]])
key_words = kw2.difference(kw1)
key_words.add('деньга')

CPU times: user 1.67 s, sys: 8.35 ms, total: 1.68 s
Wall time: 1.68 s


In [ ]:
# Сохраняем список специфических слов
with open(path + "key_words.pkl", "wb") as f:
    pickle.dump(key_words, f)

In [ ]:
# Функция для проверки списка токенов на принадлежность
def check_question(obj:list):
      amount = len(obj) if len(obj) > 0 else 1
      check = sum([1 for el in obj if el in key_words]) / amount

      if check > 0.29:
          return 1
      else:
          return 0

In [ ]:
# Проверяем работу
control = ['Мне надоело зависеть от работы!', 'Какую книгу почитать?', "Какой фильм посмотреть?", 'Главные правила богатых людей', "Что нужно знать чтобы стать богатым?", 'Кому должен подражать успешный бизнесмен?', 'Успешный бизнес - это риск.', 'Какими качествами должен обладать успешный бизнесмен?', 'Какие качества нужно развивать чтобы стать богатым?', 'Что такое деньги?', 'Можно ли быстро разбогатеть?']
result, count = [], 0
for item in control:
# for item in questions[:10000]:
  spam = preprocess_txt(item)
  result.append(check_question(spam))

for n, el in enumerate(result):
  if el == 1:
    count += 1
    print(control[n])
print(count)

Мне надоело зависеть от работы!
Главные правила богатых людей
Что нужно знать чтобы стать богатым?
Кому должен подражать успешный бизнесмен?
Успешный бизнес - это риск.
Какими качествами должен обладать успешный бизнесмен?
Что такое деньги?
Можно ли быстро разбогатеть?
8


# Сборка

In [ ]:
!pip install annoy stop_words pymorphy2 python-telegram-bot==13.3

In [ ]:
# from tqdm import tqdm_notebook, tqdm, notebook
import string
# import shutil
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import FastText
import pickle
import numpy as np
import re
import pickle
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru")).difference(['не', 'нет'])
exclude = set(string.punctuation).union(['—'])

In [ ]:
end = ['?', '.', '!', ';']
def check_sign(obj:str):
    finish = 0
    obj = ' '.join(obj.strip().split())
    for n, el in enumerate(obj[::-1]):
        if el in end:
            finish = n
            break
        else:
            continue
    return obj[:-finish] if finish > 0 else obj[:]

def preprocess_txt(line):
    # Убираем HTML тэги
    spls = re.sub(r'(<.{,4}>)', '', line)
    # Ищем цифры, разделяем их пробелами
    spls = [i + ' ' if i.isdigit() else i for i in spls.strip()]
    # Убираем знаки препинания
    spls = ''.join(i for i in spls if i not in exclude).split()
    # spls = [i.lower() for i in spls]
    # Приводим к нормальной форме
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    # Убираем стоп-слова
    spls = [i for i in spls if i not in sw and len(i) > 1]
    return spls

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/bot/'

Mounted at /content/drive/


In [ ]:
# Загрузка модели, дерева индексов, словаря индексов (Общие вопросы)
modelFT = FastText.load(path + "ft_model")
ft_index = annoy.AnnoyIndex(450, 'angular')
ft_index.load(path + 'speaker.ann')
# Словарь ответов
with open(path + 'index_speaker.pkl', 'rb') as f:
    index_map = pickle.load(f)
# Список ключевых слов
with open(path + 'key_words.pkl', 'rb') as f:
    key_words = pickle.load(f)

In [ ]:
# Загружаем предобученную модель генерации текста
model_path = '/content/drive/MyDrive/model_gpt_wealth'
tokenizer_path = '/content/drive/MyDrive/gpt_wealth'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model_g = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
def get_response_unknown():
    variant = ['Ваш вопрос непонятен.',
               'Можно сформулировать вопрос по-другому?',
               'Даже не знаю, что и сказать.',
               'Это Вы со мной разговариваете?']
    return variant[np.random.randint(0, len(variant))]

In [ ]:
# Получение ответа на общий вопрос
def get_response_general(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(450)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, include_distances=True)
    if answers[1][0] > 0.9:
        return get_response_unknown()
    else:
        return [index_map[i].strip() for i in answers[0]][np.random.randint(0, 5)]
        # return [index_map[i].strip() for i in answers[0]]
# Возможно есть смысл выбирать случайный вариант их нескольких ответов.

In [ ]:
# Получение ответа в модели генерации текста
def get_response_wealth(question, model, tokenizer):
    tokens = tokenizer(question, return_tensors='pt')
    size = tokens['input_ids'].shape[1]
    output = model.generate(**tokens, do_sample=True, max_length=size+180, repetition_penalty=3., temperature=1.05, num_beams=5,)
    # output = model.generate(**tokens, do_sample=True, max_length=size+150, repetition_penalty=5., temperature=2.999, num_beams=2,)
    decoded = tokenizer.decode(output[0])
    result = decoded[len(question):]
    # return result
    return check_sign(result)

In [ ]:
text = '.'
get_response_general(text, ft_index, modelFT, index_map)

([176, 420, 1027, 1615, 1746], [1.4142135381698608, 1.4142135381698608, 1.4142135381698608, 1.4142135381698608, 1.4142135381698608])


['Да ладно! Не кипятись! Ночная смена только собирается!',
 'кобздец, придется уйти скоро, если не отдашься, по себе знаю, не отдалась-пришлось уйти.',
 'Отчасти в этом что-то есть... но и как аксиому не следует это воспринимать.',
 'кризис - скучно.. . с этим можно бороться обоюдно желательно).',
 'реально, но не сейчас.']

In [ ]:
text = 'Алексей Мордашов'
get_response_wealth(text, model_g, tokenizer)

'— о жизненном пути олигарха, Владимире Путине и российских миллиардерах. Стоит отметить, что в отличие от большинства российских мультимиллионеров, российские мультимиллионеры относительно молоды. И по общему количеству активов (в России их уже около четырех тысяч) можно уверенно сказать, что очередные витки истории мы пережили не зря. В 1999 году исполнилось сто лет со дня рождения первого вице-премьера РФ, Виктора Черномырдина. Тогда же началась история с Вагитом Алекперовым, которому нынче уже 79 лет.'

In [ ]:
def check_question(obj:list):
      amount = len(obj) if len(obj) > 0 else 1
      check = sum([1 for el in obj if el in key_words]) / amount
      if check > 0.1: #(0.29)
          return 1
      else:
          return 0

# def get_answer(text):
#     spam = preprocess_txt(text)
#     if len(spam) < 1:
#         return get_response_unknown()
#     eggs = check_question(spam)
#     if eggs == 0:
#         answer = get_response_general(text, ft_index, modelFT, index_map)
#         # Тут надо предусмотреть нерелевантный ответ.
#     else:
#         answer = get_response_wealth(text, model_g, tokenizer)
#     return answer

In [ ]:
updater = Updater("6637934622:AAHhU0s42SFp05HcH6p-92GIDQJ48dlrPTQ", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher

In [ ]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

def textMessage(update: Update, context: CallbackContext):

    spam = preprocess_txt(update.message.text)
    if len(spam) < 2:
        update.message.reply_text(get_response_unknown())
        return
    eggs = check_question(spam)
    if eggs == 0:
        answer = get_response_general(update.message.text, ft_index, modelFT, index_map)
        update.message.reply_text(answer)
        # Тут надо предусмотреть нерелевантный ответ.
    else:
        answer = get_response_wealth(update.message.text, model_g, tokenizer)
        update.message.reply_text(answer)
    return

In [ ]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

# Start the Bot
updater.start_polling()
updater.idle()

ERROR:telegram.ext.dispatcher:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/dispatcher.py", line 442, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/handler.py", line 160, in handle_update
    return self.callback(update, context)
  File "<ipython-input-13-40960ed758c4>", line 6, in textMessage
    spam = preprocess_txt(update.message.text)
AttributeError: 'NoneType' object has no attribute 'text'
ERROR:telegram.ext.dispatcher:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/dispatcher.py", line 442, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/handler.py", line 160, in handle_update
    return self.callback(upd

In [ ]:
text = 'Как Алекперов стал миллионером?'
get_answer(text)

'(выберите свой вариант ответа) — благодаря участию в операциях по конвертации рублей в доллары; — благодаря тесным связям с правительственными кругами России, Китая и США; Россия Алекперов Вагит Юсуфович — президент ОАО "НК "ЛУКОЙЛ", владелец 75 % акций этого предприятия. Ежегодный оборот компании составляет 1,8 миллиарда долларов. Состояние оценивается в 15 миллиардов долларов - 65-е место в рейтинге Forbes! Евтушенков Владимир Петрович — вице-президент Российского Союза промышленников и предпринимателей, основатель и Председатель Совета директоров консорциума «Альфа-групп», куда входят: Альфа-банк (финансы), группа «АльфаСтрахование»;'

In [ ]:
n = 0
with open('Otvety.txt', 'r')as f:
  for line in f:
    print(line)
    n += 1
    if n > 50:
      break

In [ ]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens,
    #end_token=end_token_id,
    do_sample=True,
    max_length=size+230,
    repetition_penalty=7.,
    temperature=1.05,
    num_beams=5,)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Кто и зачем создал деньги? (выберите свой вариант ответа) — чтобы все были довольны; — чтобы каждому досталось по серьгам; — чтобы не случилось "перекоса" в сторону обогащения;
Россия
Деньги любят решительных людей. Они добиваются успеха, основываясь на собственных ошибках и достижениях. Такие люди могут стать высокооплачиваемыми сотрудниками, менеджерами, инвесторами. Их девиз: "Только оригинальная личность сможет сделать деньги".
Америка
Джон Рокфеллер-старший начал свою карьеру как агент по продаже спецпредприятий США. В конце концов, он стал известен, как человек весьма сведущий в финансовых делах. Открыл счет в банке и был зачислен в штат пожарной инспекции Нью-Йорка. Провел неделю в резерве, готовясь к вылету навстречу смертельной опасности. Взлет прошел удачно, а вот приземление -неудобно. Пришлось срочно вызывать "скорую". Дежурный врач запросил у Рокфеллера выписку из больницы. Но когда тот ознакомился с информацией, выписанной ему врачом, вынужден был отказаться от этой безум